In [2]:
import pandas as pd, numpy, scipy, matplotlib

nacitanie dat do dafatfamov

In [3]:
df_product = pd.read_csv("product.csv", sep='\t')
df_session = pd.read_csv("session.csv", sep='\t')
df_users = pd.read_csv("user.csv", sep='\t')
print(len(df_product), len(df_session), len(df_users))
# df_product.head()
# df_session.head()
# df_users.head()


14908 11039 2592


pocet zaznamov v kazdom subore

In [4]:
print(f'pocet pozretych produktov: {len(df_product)}\npocet uzivatelskych sedeni: {len(df_session)}\npocet pouzivatelov: {len(df_users)}')

pocet pozretych produktov: 14908
pocet uzivatelskych sedeni: 11039
pocet pouzivatelov: 2592


In [11]:
df_product.describe().round()

,count,mean,std,min,25%,50%,75%,max
product_ean,14908.0,5.037582e+12,2.895454e+12,554747017.0,2.509789e+12,5.071681e+12,7.565555e+12,9.999524e+12
